using the napari tracks layers : https://napari.org/stable/howtos/layers/tracks.html 


In [1]:
import napari 

In [2]:
v = napari.Viewer()

Assistant skips harvesting pyclesperanto as it's not installed.


In [3]:
import numpy as np 

In [4]:
v.add_tracks?

Signature:
v.add_tracks(
    data,
    *,
    features=None,
    properties=None,
    graph=None,
    tail_width=2,
    tail_length=30,
    head_length=0,
    name=None,
    metadata=None,
    scale=None,
    translate=None,
    rotate=None,
    shear=None,
    affine=None,
    opacity=1,
    blending='additive',
    visible=True,
    colormap='turbo',
    color_by='track_id',
    colormaps_dict=None,
    cache=True,
    experimental_clipping_planes=None,
) -> napari.layers.tracks.tracks.Tracks
Docstring:
Add a Tracks layer to the layer list. 

Parameters
----------
data : array (N, D+1)
    Coordinates for N points in D+1 dimensions. ID,T,(Z),Y,X. The first
    axis is the integer ID of the track. D is either 3 or 4 for planar
    or volumetric timeseries respectively.
features : Dataframe-like
    Features table where each row corresponds to a point and each column
    is a feature.
properties : dict {str: array (N,)}, DataFrame
    Properties for each point. Each property should be 

In [25]:
def periodic_random_walk_tracks(n_particles, n_timesteps):    
    
    positions = []

    # initialize start positions     
    positions.append(np.random.random((n_particles, 3)))
    for it in range(n_timesteps):
        d_xyz = (np.random.random((n_particles, 3)) - 0.5) * 0.1
        new_pos = positions[-1] + d_xyz 
        # 
        exited_right = new_pos > 1.0
        exited_left = new_pos < 0.0
        new_pos[exited_right] = new_pos[exited_right] - 1.0
        new_pos[exited_left] = new_pos[exited_left] + 1.0
        positions.append(new_pos)

    return positions

In [28]:
tracks = periodic_random_walk_tracks(10, 4)
tracks[0].shape

(10, 3)